# 1509 Applications of Data Science
## Movie Recommendation System Using Item Based Collaborative Filtering
Based on publications of the following Authors: Poonam Sharma, Lokesh Yadav

By Philipp Markopulos (h12030673@s.wu.ac.at)

### Process
1. Ratings Data Preprocessing (user-item-rating-matrix)
2. Keywords Data Preprocessing (item-keywords-binary-matrix)
3. Content-based filtering algorithm
6. Item-based collaborative filtering algorithm
7. Hybrid RecSys algorithm
8. Testing
10. Evaluation

In [56]:
import pandas as pd
import numpy as np
import json
import ast
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
from sklearn.metrics.pairwise import cosine_similarity
import sys
from IPython.display import clear_output
import time
import csv

In [57]:
# set a limit on how many users and movies you want to include
# at these limits(200,200) the whole code runs within few minutes
# the whole dataset runs for 1.5 days
usersLimit = 200
movieLimit = 200

In [58]:
# sparse pivot table creator
def sparsePivotTable(df, rowCol, colCol, valCol):
  # create factor arrays for the rows and columns
  rowCol_cat = CategoricalDtype(sorted(df[rowCol].unique()), ordered=True)
  colCol_cat = CategoricalDtype(sorted(df[colCol].unique()), ordered=True)

  # create the rows and columns from the respective factors
  row = df[rowCol].astype(rowCol_cat).cat.codes
  col = df[colCol].astype(colCol_cat).cat.codes
  
  # create the sparse matrix
  # 1: pass the values for the cells array
  # 2: pass the indices (row) and the names (col)
  # 3: pass the dimensions (length of row and length of col)
  sparse_matrix = csr_matrix((df[valCol], (row, col)), shape=(rowCol_cat.categories.size, colCol_cat.categories.size))
  # turn it into a sparse Pandas Dataframe
  dfs = pd.DataFrame.sparse.from_spmatrix(sparse_matrix)
  #return dfs
  return sparse_matrix

# turning a sprase pivot table into a pd.dataframe (optional)
def getSparseDf(sparse_matrix):
  return pd.DataFrame.sparse.from_spmatrix(sparse_matrix)

#### 1. Ratings Data Preprocessing
create the user-item-ratings-matrix

In [59]:
# read the data from the csv
ratings = pd.read_csv("ratings_small.csv")
ratings = ratings[ratings['userId']<usersLimit]
ratings = ratings[ratings['movieId']<movieLimit]
# call the Sparse Pivot Table Function
ratings = sparsePivotTable(ratings, "userId", "movieId", "rating").astype(int)
display(ratings.todense())
print(ratings.shape)
print("Filesize of the matrix:", sys.getsizeof(ratings)) # this is significantly smaller than creating it entirely with Pandas Pivot Tables

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

(182, 149)
Filesize of the matrix: 64


In [60]:
getSparseDf(ratings)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,...,0,0,4,0,0,0,0,0,0,3,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0
4,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,4,0,0,0,3,0,0,2,...,0,0,0,0,0,0,0,0,0,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0
178,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
179,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
180,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### 2. Keywords Data Preprocessing
get the item-keyword-binary-matrix

In [61]:
# Load the Keywords Dataset
# 1st col: movieId
# 2nd col: array of dictionaries (2 key-item pairs: keyword id, keyword name) as string
movie_keywords_list = pd.read_csv("keywords.csv")
movie_keywords_list = movie_keywords_list[movie_keywords_list['id']<movieLimit]
# turn the strings into python lists
# cannot use JSON functions, as JSON format rules are not met (single quotes instead of double quotes)
# literal_eval takes the literal meaning of the string as Python code, which results in valid lists
movie_keywords_list.keywords = movie_keywords_list.keywords.apply(lambda x: ast.literal_eval(x))
#display(movie_keywords_list)

# rather than having one row per movie with an array of keyword ids
# … we want a df with one row per keyword per movie
movie_keywords_longer = {}
j = 0
for i, row in movie_keywords_list.iterrows():
  for keyword in row.keywords:
    movie_keywords_longer[j] = {'movieId': row.id, 'keyword': keyword['id']}
    j += 1
# rows and columns are swapped, transpose it
movie_keywords_longer = pd.DataFrame(movie_keywords_longer).transpose()
# remove duplicates
movie_keywords_longer = movie_keywords_longer.drop_duplicates()
# create a column with the value 1; this will be the value for every match in the pivot table
movie_keywords_longer["match"] = 1
#display(movie_keywords_longer)

# call the Sparse Pivot Table Function
movie_keywords = sparsePivotTable(movie_keywords_longer, "movieId", "keyword", "match")
display(movie_keywords)
display(movie_keywords.todense())
print("Filesize of the matrix:", sys.getsizeof(movie_keywords))

<152x1321 sparse matrix of type '<class 'numpy.longlong'>'
	with 1917 stored elements in Compressed Sparse Row format>

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Filesize of the matrix: 64


In [62]:
# make a df with all keywords (optional)
# use list comprehension to gather all keyword dictionaries
keywords_list = [keyword for row in movie_keywords_list.keywords for keyword in row]
# make a Pandas DF from it
keywords_df = pd.DataFrame(keywords_list)
keywords_df = keywords_df.drop_duplicates()
keywords_df = keywords_df.set_index('id')
keywords_df = keywords_df.sort_index()
keywords_df

,name
id,
30,individual
65,holiday
74,germany
83,saving the world
90,paris
...,...
236189,future noir
236316,anarchic comedy
238332,animal research


#### 3. Content-based filtering algorithm
this algorithm creates a user-keyword-profile based on his likes and compares it to all films, the best matches are returned as recommendations

In [63]:
# creating user-profiles frmo their likes
def getUserProfile(ratings, userId, movieKeywordsMatrix):
  """
  Input: 
  ratings: 1-5 int user-ratings-matrix (sparse, 0 if unrated)
  userId: int of user you want the profile of
  movieKeywordsMatrix: binary movie-keyword-matrix (sparse)
  Output: 
  userProfile: binary array for user liking a keyword
  """
  userRatings = ratings[userId]
  # assume that a rating of 3 or higher means that a film is liked
  likingThreshold = 3
  
  # turn the ordinal ratings into binary to indicate liking a movie
  # this requires a dense array, which is not problematic, as we are only looking at one user profile
  movieLikes = userRatings
  movieLikes = np.array(userRatings.todense())
  movieLikes = np.where(movieLikes >= likingThreshold)[1]
  
  # get a list of ids of the movies the user liked
  movieLikeIds = [likeId for likeId, like in enumerate(movieLikes) if like]    
  # select the liked ids from the movies*keywords matrix
  likedMoviesKeywords = movieKeywordsMatrix[movieLikeIds]
  # now we have a matrix with one row for each liked movie and a binary match for each keyword
  # sum the matches vertically 
  userProfile = likedMoviesKeywords.sum(axis=0)

  # now reduce the numbers to be either 1 for liking a keyword or else 0
  userProfile = userProfile
  userProfile = (userProfile>0).astype(int)

  print("User Profile created")
  return userProfile

# Example
getUserProfile(ratings, 0, movie_keywords)

User Profile created


matrix([[0, 0, 0, ..., 0, 0, 0]])

In [64]:
# function for comparing a profile with all movies' profiles to get similarities
def getKeywordProfileSim(userProfile, movieKeywordsMatrix):
  #print("Calculating similarities")
  # loop through all movies and calculate cosine similarity to userProfile
  #sim = [(id, cosine_similarity(userProfile, filmActive)[0][0]) for id, filmActive in enumerate(movieKeywordsMatrix)]
  movie_keywords = movieKeywordsMatrix
  sim = cosine_similarity(userProfile, movie_keywords)
  return sim

# function for comparing a user-profile with all movie profiles to get similarities
def getUserMovieSimilarity(userProfile, movieKeywordsMatrix):
  # reshape the user profile to be a 1d matrix (necessary to get cosine similarity)
  userProfile = np.array(userProfile)
  sim = getKeywordProfileSim(userProfile, movieKeywordsMatrix)
  return sim

# Example
user = getUserProfile(ratings, 69, movie_keywords)
getUserMovieSimilarity(user, movie_keywords)

User Profile created


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]])

In [85]:
# perform content based filtering to get n recommendations for a user
def contentBasedFiltering(ratingsArr, userId, movieKeywords, n=10):
  """
  returns top n movies to recommend and their similarity to the user profile
  """
  # get a userProfile(binary array. 1 indicates the user likes the keyword)
  userProfile = getUserProfile(ratingsArr, userId, movieKeywords)
  # get the similarities between each movie and the user profile
  simRowCos = getKeywordProfileSim(userProfile, movieKeywords)
  # get the ids of the top similarities
  topSimilarMovieIds = np.argsort(simRowCos)[0][::-1][:n]
  topSims = simRowCos[0][topSimilarMovieIds]
  return [(id, simRowCos[0][id]) for id in topSimilarMovieIds]
  
# Example
userId = 1
contentBasedFilteringRec = contentBasedFiltering(ratings, userId, movie_keywords)
print("Content-based recommendations for user", userId)
contentBasedFilteringRec

User Profile created
Content-based recommendations for user 1


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:590: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


[(9, 0.36985463743231434),
 (11, 0.35020209798623764),
 (12, 0.3221390769615825),
 (7, 0.314732590898345),
 (15, 0.3071475584169757),
 (6, 0.2993704085400459),
 (4, 0.27472112789737807),
 (10, 0.2569780843750234),
 (5, 0.23791547571544328),
 (8, 0.21718612138153465)]

#### 4. Item-based collaborative filtering algorithm


In [66]:
# getting item-based similarities between all items 
# transpose the matrix before to allow for column-wise calcuations
rating_movie_similarities = cosine_similarity(ratings.transpose())

In [67]:
# function for getting the ids of the movies most similar to the active one
def getMostSimilarMovie(movieId, similarities):
  # get the similarities of that particular movie
  similarities = similarities[movieId]
  similarities = np.delete(similarities, movieId)
  mostSimilarId = np.argmax(similarities)
  return (mostSimilarId, similarities[mostSimilarId])

In [88]:
# getting the best n recommendations for a user

def getIBCF(userId, ratingsMatrix, similaritiesMatrix = "nothing", n=10):
  # this should not be done everytime the function is called
  # but in case someone doesnt compute the similarities beforehand, we do it here
  if similaritiesMatrix == "nothing":
    # getting similarities between ratings 
    # transpose the matrix before to allow for column-wise calcuations
    similaritiesMatrix = cosine_similarity(ratingsMatrix.transpose())
  # get the sparse ratings for the user
  userRatings = ratings[userId,:].todense()
  # get the indices of the rated items
  userRatingIds = userRatings.nonzero()
  # make get a matrix with the indices and the corresponding ratings
  userRating = np.array([
      userRatingIds[1],
      np.array(userRatings[userRatingIds])[0]
      ])
  # create an argsort array; this returns the indices to grab to get the array in order
  sortBy = np.argsort(userRating[1])
  sortBy = np.flip(sortBy)
  # recreate the 2d-array, but use the sorted indices
  userRating =userRating[0][sortBy]
  # we need integer indices for indexing later …
  userRating = userRating.astype(int)
  # get the most similar movies for the top n rated movies
  itemBasedCollabFilteringRec = [getMostSimilarMovie(movieId, rating_movie_similarities) for movieId in userRating[:n]]
  return itemBasedCollabFilteringRec

# Example
userId = 1
print("Item-based recommendations for user", userId)
getIBCF(userId, ratings, rating_movie_similarities)

Item-based recommendations for user 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


[(121, 0.6074020839153171),
 (107, 0.5387568739348635),
 (52, 0.5274466105856278),
 (9, 0.6413226519594036),
 (121, 0.5618250098679385),
 (44, 0.5742216204701542),
 (31, 0.6008398987548947),
 (121, 0.6418729336609709),
 (124, 0.545858436324737),
 (9, 0.6339348756886364)]

#### Predicting movie ratings using IBCF
$predictedRating(u_x, i_y)= Σ^i_0sim(ratings[,y], ratings[,i])*ratings[x,y]$

In [69]:
# predicting a movieRating for a user for a certain movie

def predictMovieRatingIBCF(userId, movieId, ratingsMatrix, similaritiesMatrix, n=10):
  # get the similarities of the movies compared to the current movie
  sim = similaritiesMatrix[movieId]
  # get the ids of the n most similar movies
  topSimIds = np.where(np.argsort(sim)<n)
  # get the similarities of the top similar movies
  topSim = sim[topSimIds]
  # normalize their similarities to get the movies ratings weights
  topSimNorm = topSim / sum(topSim)
  # get the ratings of the top n similar movies that the rated
  topSimRatings = ratings[userId].todense()[0,[topSimIds]]
  topSimRatings = np.array(topSimRatings)[0][0]
  # mulitply their ratings with their normalized similarities
  weightedRatings = topSimNorm * topSimRatings
  # sum it up and thats you predicted rating for the movie
  predicTedRating = sum(weightedRatings)
  return round(predicTedRating)

# Example for user 10 and movie 10
predictMovieRatingIBCF(10, 10, ratings, rating_movie_similarities)

1

#### 5. Hybrid RecSys Algorithm
Content-Based -> fill sparse ratings matrix with pseudo-ratings

Item-Based Collaborative Filtering -> replace pseudo-ratings with proper rating predictions


In [70]:
# function for creating a pseudo ratings matrix

def getUserPseudoRatings(userId, ratingsMatrix):
  start_time = time.time()
  # select the user's row
  userRow = ratingsMatrix.todense()[userId]
  userRow = np.array(userRow)[0]
  # get the ids of the rated and unrated movies
  ratedIds = userRow != 0
  unratedIds = userRow == 0
  # for each film he hasnt rated
  for unratedFilmId,_ in enumerate(userRow[unratedIds]):
    unratedFilmProfile = movie_keywords[unratedFilmId]
    # get similarities to all films he rated
    sim = getKeywordProfileSim(unratedFilmProfile, movie_keywords[ratedIds])
    sim = sim[0]
    # normalize the similarities to add up to 1
    sim = sim / sum(sim)
    # multiply each normalized similarity with the rating of the movie you compared the unrated movie to
    weightedRatings = userRow[ratedIds] * sim
    # sum up the weighted ratings and round the result
    pseudoRating = sum(weightedRatings)
    # if the sum is np.nan, you cannot round it, just put it to 0
    if np.isnan(pseudoRating):
      pseudoRating = 0
    pseudoRating = round(pseudoRating)
    # save the pseudoRating to the userRow 
    try:
      userRow[userId] = pseudoRating
    except:
      continue
  clear_output()
  print(f"User {userId} done –", time.time() - start_time, "seconds taken")
  #print(userRow)
  ratingsMatrix[0] = userRow
  return ratingsMatrix


In [71]:
# create a copy of the user-item-ratings-matrix to fill in th pseudo ratings
pseudoRatings = ratings.copy()

# loop through the rows of the pseudo-ratings-matrix 
# this way each row is replaced with the content-based predicted ratings 
for i, row in enumerate(pseudoRatings):
  pseudoRatings = getUserPseudoRatings(i, pseudoRatings)

print(pseudoRatings.todense())
print("Filesize of the matrix:", sys.getsizeof(pseudoRatings))



User 181 done – 0.18266844749450684 seconds taken
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Filesize of the matrix: 64


#### 6. Testing
to test the hybrid algorithm, we predict some already known ratings and get a confusion matrix

now that we have a full pseudo-ratings matrix, we can perform more informed collaborative filtering predictions

In [99]:
# function for making rating predictions of a user and a movie

def getHybridRecommendation(userId, movieId, ratingsMatrix, pseudoRatingsMatrix, rating_movie_similarities):
  if ratingsMatrix[userId, movieId]:
    print("Movie already rated")
    return ratingsMatrix[userId, movieId]
  else:
    predictedRating = predictMovieRatingIBCF(userId, movieId, pseudoRatingsMatrix, rating_movie_similarities)
    return predictedRating

# Example
userId = 1
movieId = 9
rec = getHybridRecommendation(userId, movieId, ratings, pseudoRatings, rating_movie_similarities)
print("Hybrid Recommendation")
print(f"Rating for user {userId} and movie {movieId} predicted:", rec)

Movie already rated
Hybrid Recommendation
Rating for user 1 and movie 9 predicted: 4


In [73]:
# create a copy of the ratings and predict a rating for every unrated movie
ratings_test = ratings.copy()

for i,row in enumerate(ratings_test):
  if i % 500 == 0: print(f"{i} tested")
  # make the row an accessible array
  row = np.array(row.todense())[0]
  # get the ids where the ratings are known
  ratedIds = np.where(row != 0)[0]
  # overwrite the ratings to be 0
  ratings_test[ratedIds] = 0
  # now predict the rating for every previously known rating
  for j in ratedIds:
    ratings_test[i, j] = predictMovieRatingIBCF(i, j, pseudoRatings, rating_movie_similarities)

clear_output()

In [74]:
print(ratings_test.todense())
print(sys.getsizeof(ratings_test))
print(ratings_test.shape)
print(ratings.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
64
(182, 149)
(182, 149)


#### 7. Evaluation
to test the hybrid algorithm, we predict some already known ratings and measure the metrics accuracy, precision and recall

In [75]:
# create functions to extract accuracy, precision and recall from matrices of predictions

def total(x):
  return sum(sum(x))

def truePositives(x,y): # x:actual, y: predicted
  tp = np.logical_and(x.copy(), y.copy())
  return tp

def trueNegatives(x,y): # x:actual, y: predicted
  pn = np.logical_not(y.copy())
  nn = np.logical_not(x.copy(), y.copy())
  tn = np.logical_and(pn, nn)
  return tn

def falsePositives(x,y): # x:actual, y: predicted
  n = np.logical_not(x.copy())
  fp = np.logical_and(y.copy(), n)
  return fp

def falseNegatives(x,y): # x:actual, y: predicted
  pn = np.logical_not(y.copy())
  fn = np.logical_and(pn, x.copy())
  return fn

def accuracy(x,y): # x:actual, y: predicted
  return (total(truePositives(x,y)) + total(trueNegatives(x,y))) / (total(x == x))

def precision(x,y): # x:actual, y: predicted
  return total(truePositives(x,y)) / (total(truePositives(x,y)) + total(falsePositives(x,y)))

def recall(x,y): # x:actual, y: predicted
  return total(truePositives(x,y)) / (total(truePositives(x,y)) + total(falseNegatives(x,y)))

In [76]:
# convert the ratings and predicted ratings to binary (like, not like)
likes = ratings.copy() >= 3
likes = np.array(likes.todense())
likes_test = ratings_test.copy() >= 3
likes_test = np.array(likes_test.todense())

In [77]:
print("Accuracy:", accuracy(likes, likes_test))
print("Precision:", precision(likes, likes_test))
print("Recall:", recall(likes, likes_test))

Accuracy: 0.9773950881333432
Precision: 0.9989247311827957
Recall: 0.6028552887735237


What do the results suggest:
- We can see that there is generally a high precision, with only a few "bad" movies being suggested
- Unfortunately tough, we also encounter low recall with lots of good movies not being suggested by the algo.
- Still, I personally would prefer this approach with few results, with high precision, rather than vice versa: many results with low precision